# Twitter Sentiment Analysis 

In [1]:
# data link: https://raw.githubusercontent.com/laxmimerit/twitter-data/master/twitt30k.csv

In [2]:
import numpy as np
import pandas as pd

In [3]:
from sklearn.model_selection import train_test_split
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

from sklearn.feature_extraction.text import TfidfVectorizer

In [4]:
df = pd.read_csv('https://raw.githubusercontent.com/laxmimerit/twitter-data/master/twitt30k.csv')
df

,twitts,sentiment
0,@robbiebronniman Sounds like a great night.,1
1,Damn the person who stolde my wallet !!!!! Ma...,1
2,Greetings from the piano bench (photo) http:/...,1
3,@drewryanscott i love it!! i love you!! haha f...,1
4,"@kissthestars Pretty pretty pretty please, pak...",0
...,...,...
29995,@Calumfan1 is it in any way related to photosh...,0
29996,@Swiz_NZ really? wow thats crap,0
29997,"At the 2010 lexus HS250h press event. Again, ...",0
29998,@karmicunderpath ooooh now there's a nice thou...,1


In [5]:
df['sentiment'].value_counts()

0    15000
1    15000
Name: sentiment, dtype: int64

## SVM Model and Data Preparation 

In [6]:
def run_svm(df):
    X = df['twitts']
    y = df['sentiment']

    tfidf = TfidfVectorizer()
    X = tfidf.fit_transform(X)

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0, stratify = y)

    print('shape of X: ', X.shape)

    clf = LinearSVC()
    clf.fit(X_train, y_train)

    y_pred = clf.predict(X_test)

    print()
    print('Printing Report')
    print(classification_report(y_test, y_pred))
    
    return tfidf, clf

In [7]:
%%time
tfidf, clf = run_svm(df)

shape of X:  (30000, 40854)

Printing Report
              precision    recall  f1-score   support

           0       0.75      0.74      0.75      3000
           1       0.74      0.75      0.75      3000

    accuracy                           0.75      6000
   macro avg       0.75      0.75      0.75      6000
weighted avg       0.75      0.75      0.75      6000

CPU times: user 536 ms, sys: 14.1 ms, total: 550 ms
Wall time: 548 ms


In [8]:
x = ['i am really happy. thanks a lot for coming with me']

In [9]:
clf.predict(tfidf.transform(x))

array([1])

## Data Cleaning and Retraining SVM 

In [10]:
# Use our preprocess python package

In [11]:
import preprocess_kgptalkie as pp

ModuleNotFoundError: No module named 'preprocess_kgptalkie'

In [ ]:
pp.__version__

In [ ]:
df['twitts'] = df['twitts'].apply(lambda x: x.lower())

In [ ]:
df['twitts'] = df['twitts'].apply(lambda x: pp.cont_exp(x))

In [ ]:
df

In [ ]:
run_svm(df)

In [ ]:
# remove emails and urls

df['twitts'] = df['twitts'].apply(lambda x: pp.remove_emails(x))
df['twitts'] = df['twitts'].apply(lambda x: pp.remove_urls(x))
df['twitts'] = df['twitts'].apply(lambda x: pp.remove_rt(x))
df['twitts'] = df['twitts'].apply(lambda x: pp.remove_html_tags(x))
df['twitts'] = df['twitts'].apply(lambda x: pp.remove_special_chars(x))


In [ ]:
tfidf, clf = run_svm(df)

In [ ]:
x

In [ ]:
clf.predict(tfidf.transform(x))

## Fine Tuning Model

In [ ]:
df

In [ ]:
def run_svm(df):
    X = df['twitts']
    y = df['sentiment']

    tfidf = TfidfVectorizer(norm = 'l1', ngram_range=(1,2), analyzer='word', max_features=5000)
    X = tfidf.fit_transform(X)

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0, stratify = y)

    print('shape of X: ', X.shape)

    clf = LinearSVC()
    clf.fit(X_train, y_train)

    y_pred = clf.predict(X_test)

    print()
    print('Printing Report')
    print(classification_report(y_test, y_pred))
    
    return tfidf, clf

run_svm(df)

## Saving and Loading ML Model 

In [ ]:
import pickle

In [ ]:
pickle.dump(clf, open('clf.pkl', 'wb'))
pickle.dump(tfidf, open('tfidf.pkl', 'wb'))

In [ ]:
del clf
del tfidf

In [ ]:
clf = pickle.load(open('clf.pkl', 'rb'))
tfidf = pickle.load(open('tfidf.pkl', 'rb'))

In [ ]:
clf

In [ ]:
# tfidf.vocabulary_

In [ ]:
x

In [ ]:

clf.predict(tfidf.transform(x))

# Real-Time Twitter Sentiment Analysis 

In [ ]:
consumer_key = 'R7DGimRNkT11sbngA0MRqLmNE'
consumer_secret = 'w5Axtw43feejwgmPIhqPhPOt1aHso1Guw1yuFwlmijtlh0vguK'
access_token = '1279486577656295425-l3gaKqKuHQdKl44rPXUc0WYcc26wgq'
access_token_secret = '80dGAdcx6LuoWM1mSt669V5NESP0EOuX1dK8Mianjqxi2'

In [ ]:
# !pip install tweepy

In [ ]:
import tweepy

auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)

api = tweepy.API(auth)

public_tweets = api.home_timeline()


In [ ]:
type(public_tweets)

In [ ]:
public_tweets[0].text

In [ ]:
for tweet in public_tweets:
    print(tweet.text)

## Tracking Keywords on Twitter 

In [ ]:
import json
import pickle
import tweepy
import csv

In [ ]:
from textblob import TextBlob

In [ ]:
import preprocess_kgptalkie as pp

In [ ]:
clf = pickle.load(open('clf.pkl', 'rb'))
tfidf = pickle.load(open('tfidf.pkl', 'rb'))

In [ ]:
def predict_sentiment(x):
    x = [x]
    sent = clf.predict(tfidf.transform(x))
    return sent

In [ ]:
predict_sentiment(x)

In [ ]:
predict_sentiment('what i am not the best')[0]

In [ ]:
track_keyword = ['usa', 'china']

In [ ]:
usa = 0
china = 0

with open('sentiment.csv', 'w') as file:
    writer = csv.DictWriter(file, fieldnames=track_keyword)
    writer.writeheader()


class MyStreamListener(tweepy.StreamListener):
    
    def on_status(self, status):
        print(status.text)
    
    def on_data(self, data):
        raw_twitts = json.loads(data)
        try:
            x = str(raw_twitts['text']).lower()
            x = pp.cont_exp(x)
            x = pp.remove_emails(x)
            x = pp.remove_html_tags(x)
            x = pp.remove_rt(x)
            x = pp.remove_special_chars(x)
            x = pp.remove_urls(x)
            
#             blob = TextBlob(x)
#             sentiment = predict_sentiment(x)[0]
#             print(sentiment)
            
            global china
            global usa
            
            if 'usa' in x and 'china' not in x:
                sent = predict_sentiment(x)[0]
                usa = usa + sent
                
            elif 'china' in x and 'usa' not in x:
                sent = predict_sentiment(x)[0]
                china = china + sent
                
            else:
                pass
            
            print('usa: ', usa, 'china: ', china)
            
            with open('sentiment.csv', 'a') as file:
                writer = csv.DictWriter(file, fieldnames=track_keyword)
                info = {
                    'usa': usa,
                    'china': china
                }
                writer.writerow(info)
            
            
        except:
            pass
        
        
        
    def on_error(self, status_code):
        if status_code == 420:
            print('Error 420')
            #returning False in on_error disconnects the stream
            return False

In [ ]:
myStreamListener = MyStreamListener()
myStream = tweepy.Stream(auth = api.auth, listener=myStreamListener)

In [ ]:
myStream.filter(track=track_keyword)